In [4]:
import os, sys

notebook_dir = os.getcwd()

#add path
sys.path.append(os.path.abspath(os.path.join(notebook_dir, '..')))

In [2]:
from scripts.preprocess_data import *

In [3]:
preprocess_data("../telegram_messages.json")

Preprocessed data saved to preprocessed_data.csv


In [7]:
from scripts.CoNLL_formatting import *

ModuleNotFoundError: No module named 'scripts.CoNLL_formatting'

In [3]:
# Read the data and save the subset_df in a variable
data = 'preprocessed_data.csv'
subset_df = read_subset_df(data)

# Call the label_data function to process and save the labeled data
label_data(subset_df)

         channel         sender                  timestamp  \
0  @ZemenExpress -1001307493052  2025-03-08 13:58:44+00:00   
1  @ZemenExpress -1001307493052  2025-03-08 11:35:13+00:00   
2  @ZemenExpress -1001307493052  2025-03-08 11:34:52+00:00   
3  @ZemenExpress -1001307493052  2025-03-07 13:33:41+00:00   
4  @ZemenExpress -1001307493052  2025-03-07 13:33:40+00:00   

                                             content  \
0  💥💥...............🌞.................💥💥\n\n❓ በረፍ...   
1  💥💥....................................💥💥\n\n📌L...   
2  💥💥....................................💥💥\n\n📌L...   
3                                                NaN   
4                                                NaN   

                                              tokens  
0  ['💥💥...............🌞.................💥💥', '❓',...  
1  ['💥💥....................................💥💥', '...  
2  ['💥💥....................................💥💥', '...  
3                                                 []  
4              

In [6]:
from scripts.load_conll_dataset import *
from scripts.tokenization_alignment import *
from scripts.training_and_tuning import *
from transformers import AutoTokenizer, AutoModelForTokenClassification
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'scripts.load_conll_dataset'

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access the variables for api
token = os.getenv('TOKEN')

In [1]:
# Load the connl data that we made earlier
file_path = 'labeled_data.conll'
sentences, labels = load_conll_dataset(file_path)

print(f"Number of sentences: {len(sentences)}")
print(f"Number of labels: {len(labels)}")

# Optionally, print a sample to verify
for i in range(min(5, len(sentences))):  # Print up to 5 samples
    print(f"Sentence {i+1}: {sentences[i]}")
    print(f"Labels {i+1}: {labels[i]}")

NameError: name 'load_conll_dataset' is not defined

In [ ]:
# execute_codes()
# sentences, labels = load_conll_dataset('labeled_data.conll')

# Define label mapping before loading the model
label_set = set(label for label_list in labels for label in label_list)
label_map = {label: i for i, label in enumerate(label_set)}

In [ ]:
# Load the tokenizer and model
# rasyosef/bert-tiny-amharic
model_name = "rasyosef/bert-tiny-amharic"  # Change to "bert-tiny-amharic" or "afroxmlr" as needed
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=token)
model = AutoModelForTokenClassification.from_pretrained(model_name, use_auth_token=token)

In [ ]:
# from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
# from sklearn.model_selection import train_test_split

# Assuming sentences and labels are already loaded
print(f"Number of sentences: {len(sentences)}")
print(f"Number of labels: {len(labels)}")

# Ensure they are not empty before proceeding
if len(sentences) == 0 or len(labels) == 0:
    raise ValueError("Sentences or labels are empty. Please check your data loading process.")

# Use all data for training if there's only one sentence
if len(sentences) < 2:
    print("Not enough data to split into training and validation sets. Using all data for training.")
    train_sentences = sentences
    train_labels = labels
    val_sentences = []
    val_labels = []
else:
    # Split the dataset into training and validation sets
    train_sentences, val_sentences, train_labels, val_labels = train_test_split(
        sentences, labels, test_size=0.1, random_state=42
    )

# Tokenize and align labels for train and validation sets
train_tokenized = tokenize_and_align_labels(train_sentences, train_labels, tokenizer, label_map)

# If there are validation sets, tokenize those as well
if val_sentences:
    val_tokenized = tokenize_and_align_labels(val_sentences, val_labels, tokenizer, label_map)
else:
    val_tokenized = None

# Create a dictionary for the tokenized dataset
tokenized_dataset = {
    'train': train_tokenized,
    'validation': val_tokenized
}

print("Tokenized training dataset:")
print(tokenized_dataset['train'])
print("Keys in training dataset:", tokenized_dataset['train'].keys())

num_examples = len(tokenized_dataset['train']['input_ids'])
print(f"Number of examples in the training dataset: {num_examples}")

for key in tokenized_dataset['train'].keys():
    print(f"Length of {key}: {len(tokenized_dataset['train'][key])}")

for i in range(num_examples):
    if (len(tokenized_dataset['train']['input_ids'][i]) == 0 or
        len(tokenized_dataset['train']['labels'][i]) == 0):
        print(f"Empty entry found in index {i}: input_ids length: {len(tokenized_dataset['train']['input_ids'][i])}, "
              f"labels length: {len(tokenized_dataset['train']['labels'][i])}")
# Ensure the labels are not empty
if 'labels' not in tokenized_dataset['train'] or not tokenized_dataset['train']['labels']:
    raise ValueError("Labels cannot be empty.")

# Check for length mismatches in the tokenized dataset
for i in range(len(tokenized_dataset['train']['input_ids'])):
    if (len(tokenized_dataset['train']['input_ids'][i]) !=
        len(tokenized_dataset['train']['token_type_ids'][i]) or
        len(tokenized_dataset['train']['attention_mask'][i]) !=
        len(tokenized_dataset['train']['labels'][i])):
        print(f"Length mismatch at index {i}: "
              f"{len(tokenized_dataset['train']['input_ids'][i])}, "
              f"{len(tokenized_dataset['train']['token_type_ids'][i])}, "
              f"{len(tokenized_dataset['train']['attention_mask'][i])}, "
              f"{len(tokenized_dataset['train']['labels'][i])}")

# Ensure lengths are consistent
for i in range(len(tokenized_dataset['train']['input_ids'])):
    input_len = len(tokenized_dataset['train']['input_ids'][i])
    token_type_len = len(tokenized_dataset['train']['token_type_ids'][i])
    attention_mask_len = len(tokenized_dataset['train']['attention_mask'][i])
    labels_len = len(tokenized_dataset['train']['labels'][i])
    
    if not (input_len == token_type_len == attention_mask_len == labels_len):
        print(f"Length mismatch at index {i}: input_ids({input_len}), "
              f"token_type_ids({token_type_len}), "
              f"attention_mask({attention_mask_len}), "
              f"labels({labels_len})")
    else:
        print(f"Index {i} is consistent: input_ids({input_len}), "
              f"token_type_ids({token_type_len}), "
              f"attention_mask({attention_mask_len}), "
              f"labels({labels_len})")
        
# Proceed with fine-tuning if everything is correct
trainer = fine_tune_model(tokenized_dataset['train'], val_tokenized, model, tokenizer)

# If you have a validation set, you can still evaluate it later
if val_tokenized:
    eval_results = evaluate_model(trainer)
    print(eval_results)

In [ ]:
# Tokenize and align labels
tokenized_dataset = tokenize_and_align_labels(sentences, labels, tokenizer, label_map)
print(tokenized_dataset)

# # Fine-tune the model
# trainer = fine_tune_model(tokenized_dataset['train'], tokenized_dataset['validation'], model, tokenizer)

# # Evaluate the model
# eval_results = evaluate_model(trainer)
# print(eval_results)

In [ ]:
# Save the fine-tuned model and tokenizer
save_model(trainer, "fine_tuned_ner_model", tokenizer)